<a href="https://colab.research.google.com/github/songqsh/foo1/blob/master/src/hw_wk5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd

In [10]:
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

Ex 1: Solve the following linear system
$$
\begin{array}
{ll}
-5 x_1 - 7 x_2 - 7 x_3 & = -3 \\
-8 x_1 + 9 x_2 + 7 x_3 & = 5\\
-6 x_1 + 8 x_2 - 4 x_3 & = -3
\end{array}
$$

In [11]:
#matrix input
M = np.array([[-5.,-7,-7,-3.],
              [-8., 9, 7, 5], 
              [-6., 8, -4, -3]])

In [12]:
pivot(M,1,1)
M

array([[-11.22, 0.00, -1.56, 0.89],
       [-0.89, 1.00, 0.78, 0.56],
       [1.11, 0.00, -10.22, -7.44]])

In [13]:
pivot(M,0,0)
M

array([[1.00, -0.00, 0.14, -0.08],
       [0.00, 1.00, 0.90, 0.49],
       [0.00, 0.00, -10.38, -7.36]])

In [14]:
pivot(M,2,2)
M

array([[1.00, 0.00, 0.00, -0.18],
       [0.00, 1.00, 0.00, -0.15],
       [-0.00, -0.00, 1.00, 0.71]])

Ex 2. Solve the follwoing LP with tableau method
$$
\begin{array}
{ll}
\max z &  = 3 x_1 + 2 x_2 \\
\hbox{ s.t. }
\\
2x_1 + x_2 &\le 100 \\
x_1 + x_2 & \le 80\\
x_1 & \le 40 \\
x_1, x_2 \ge 0
\end{array}
$$

In [15]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

In [33]:
def minratio(ratio_list):
  pivot_row = 0;
  minratio = np.inf;
  for i in range(len(ratio_list)):
    if ratio_list[i]>0 and ratio_list[i]<minratio:
      pivot_row = i+1
      minratio = ratio_list[i]
  if minratio is np.inf:
    return 0 #no leaving variable
  else:
    return pivot_row


In [34]:
def newpivot(M): #M is the tableau
  m, n = M.shape
  m = m-1 #number of constraints in the augmented form
  n = n-2 #number of variables in the augmented form
  optimal_test = min(M[0, range(1,n+1)])
  
  if optimal_test<0:
    pivot_col = np.argmin(M[0,range(1,n+1)])+1
  else:
    print(f'pass the optimal test')
    return(0)
  ratio_list = np.divide(M[range(1,m+1),-1], M[range(1, m+1),pivot_col])

  minratiotest = minratio(ratio_list)
  if minratiotest is 0:
    print(f'no leaving variable, here is the ratio list {ratio_list}')
    return(0)
  else:
    pivot_row = minratiotest
  return(pivot_row, pivot_col)


In [35]:
# input initial tableau
M = np.array([
               [1, -3, -2, 0, 0, 0, 0], 
               [0, 2, 1, 1, 0, 0, 100], 
               [0, 1, 1, 0, 1, 0, 80],
               [0, 1, 0, 0, 0, 1, 40]
               ], dtype = float)
m, n = M.shape
m = m-1
n = n-2
print(f'{m} constraints and {n} variables')
pd.DataFrame(M) # print matrix

3 constraints and 5 variables


,0,1,2,3,4,5,6
0,1.0,-3.0,-2.0,0.0,0.0,0.0,0.0
1,0.0,2.0,1.0,1.0,0.0,0.0,100.0
2,0.0,1.0,1.0,0.0,1.0,0.0,80.0
3,0.0,1.0,0.0,0.0,0.0,1.0,40.0


In [36]:
npivot = newpivot(M)
while npivot != 0:
  print(f'new pivot is {npivot}')
  if npivot is not 0:
    pivot(M, npivot[0], npivot[1])
    print(pd.DataFrame(M))
    print(f'=======================')
    npivot = newpivot(M)

new pivot is (3, 1)
     0    1    2    3    4    5      6
0  1.0  0.0 -2.0  0.0  0.0  3.0  120.0
1  0.0  0.0  1.0  1.0  0.0 -2.0   20.0
2  0.0  0.0  1.0  0.0  1.0 -1.0   40.0
3  0.0  1.0  0.0  0.0  0.0  1.0   40.0
new pivot is (1, 2)
     0    1    2    3    4    5      6
0  1.0  0.0  0.0  2.0  0.0 -1.0  160.0
1  0.0  0.0  1.0  1.0  0.0 -2.0   20.0
2  0.0  0.0  0.0 -1.0  1.0  1.0   20.0
3  0.0  1.0  0.0  0.0  0.0  1.0   40.0
new pivot is (2, 5)
     0    1    2    3    4    5      6
0  1.0  0.0  0.0  1.0  1.0  0.0  180.0
1  0.0  0.0  1.0 -1.0  2.0  0.0   60.0
2  0.0  0.0  0.0 -1.0  1.0  1.0   20.0
3  0.0  1.0  0.0  1.0 -1.0  0.0   20.0
pass the optimal test


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
